## Scraping Auchan

Pour le site d'Auchan, nous avons effectué un scraping des produits disponibles afin de récupérer des informations détaillées sur les produits.

### Étapes principales :
1. Initialisation du **WebDriver** avec Selenium pour naviguer sur le site.
2. Sélection de la localisation (**Strasbourg**) pour adapter les produits disponibles.
3. Parcours des différentes pages de promotions.
4. Extraction des informations suivantes pour chaque produit :
   - **Nom**
   - **Prix**
   - **Unité** (ex : kg, L, etc.)
   - **Pourcentage de réduction**
5. Sauvegarde des données dans un fichier **CSV** pour une analyse ultérieure.

Le processus assure une récupération complète et organisée des promotions pour une localisation spécifique.


In [23]:
import pandas as pd
import selenium
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# Initialiser le navigateur Web
driver = webdriver.Chrome()

# Accéder au site web d'Auchan
driver.get('https://www.auchan.fr/boutique/promos')  # Remplacez par l'URL exacte du site Auchan

# Attendre que le bouton "Rejeter les cookies" soit visible et cliquer dessus
try:
    reject_cookies_button = WebDriverWait(driver, 40).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Continuer sans accepter')]"))
    )
    reject_cookies_button.click()
    print("Cookies rejetés.")
except Exception as e:
    print("Pas de bouton de rejet des cookies trouvé : ", e)
    

# Wait until the element is clickable
afficher_prix_button = WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "product-thumbnail__see-prices-button"))
)

driver.execute_script("arguments[0].scrollIntoView(true);", afficher_prix_button)
time.sleep(1)  # Optionally wait a bit after scrolling

driver.execute_script("arguments[0].click();", afficher_prix_button)


# Attendre que la barre de recherche soit visible
search_bar = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "input.shadow--light.journey__search-input.journeySearchInput"))
)

# Taper "67000" dans la barre de recherche
search_bar.send_keys("67000")

# Attendre que les suggestions apparaissent et sélectionner "Strasbourg"
suggestions_list = WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul[role='listbox'] li"))
)

for suggestion in suggestions_list:
    if "Strasbourg" in suggestion.text:
        suggestion.click()
        break

# Pause pour attendre le chargement de la page suivante
time.sleep(5)


# Recherche de l'élément contenant "Auchan Click&collect Supermarché G.G. Les Halles - Strasbourg"
try:
    store_to_select = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Auchan Click&collect Supermarché G.G. Les Halles - Strasbourg')]"))
    )   
    # Scroller jusqu'à cet élément
    driver.execute_script("arguments[0].scrollIntoView(true);", store_to_select)

    # Chercher directement le bouton "Choisir" associé, en partant de l'élément textuel et descendant dans la hiérarchie
    choose_button = store_to_select.find_element(By.XPATH, ".//following::button[contains(@class, 'btnJourneySubmit')]")
    
    # Cliquer sur le bouton "Choisir"
    driver.execute_script("arguments[0].click();", choose_button)
    
    print("Bouton 'Choisir' cliqué.")
except Exception as e:
    print(f"Erreur lors de la sélection du magasin : {e}")


Cookies rejetés.
Bouton 'Choisir' cliqué.


In [27]:
# Listes pour stocker les données
product_names = []
product_prices = []
product_units = []
discount_percentages = []

# URL de base pour la pagination
base_url = "https://www.auchan.fr/boutique/promos?page="

# Initialiser le numéro de page
page_num = 1

# Boucle pour scraper plusieurs pages
while True:
    print(f"Scraping page {page_num}")
    # Charger la page avec le numéro actuel
    driver.get(base_url + str(page_num))

    # Attendre que la page charge correctement
    time.sleep(5)

    # Trouver les annonces de produits
    products = driver.find_elements(By.XPATH, "//article[contains(@class, 'product-thumbnail')]")
    
    # Si aucun produit n'est trouvé, arrêter la boucle (fin de la pagination)
    if not products:
        print("Aucun produit trouvé, fin de la pagination.")
        break

    print(f"Nombre de produits trouvés sur la page {page_num}: {len(products)}")

    # Parcourir chaque produit pour extraire les informations
    for product in products:
        try:
            # Récupérer le nom du produit
            try:
                name_element = product.find_element(By.XPATH, ".//p[contains(@class, 'product-thumbnail__description')]")
                product_names.append(name_element.text.strip())
            except Exception as e:
                product_names.append(None)
                print(f"Erreur lors de la récupération du nom : {e}")

            # Récupérer le prix du produit
            try:
                price_element = product.find_element(By.XPATH, ".//span[contains(@data-offer-type, 'DEFAULT')]")
                product_prices.append(price_element.text.strip())
            except Exception as e:
                product_prices.append(None)
                print(f"Erreur lors de la récupération du prix : {e}")

            # Récupérer l'unité du produit
            try:
                unit_element = product.find_element(By.XPATH, ".//span[@class='product-attribute']")
                product_units.append(unit_element.text.strip())
            except Exception as e:
                product_units.append(None)
                print(f"Erreur lors de la récupération de l'unité : {e}")

            # Récupérer la réduction (si présente)
            try:
                discount_element = product.find_element(By.XPATH, ".//div[contains(@class, 'product-discount')]//span[contains(@class, 'product-discount--sticker')]")
                discount_percentages.append(discount_element.text.strip())
            except Exception:
                discount_percentages.append(None)  # Pas de réduction pour ce produit
        except Exception as e:
            print(f"Erreur lors de l'extraction d'un produit : {e}")

    # Passer à la page suivante
    page_num += 1

# Sauvegarder les données extraites dans un fichier CSV
with open('extracted_products.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Nom", "Prix", "Unité", "Réduction"])
    for i in range(len(product_names)):
        writer.writerow([product_names[i], product_prices[i], product_units[i], discount_percentages[i]])

print("Extraction terminée et sauvegardée dans 'extracted_products.csv'")
driver.quit()


Scraping page 1
Nombre de produits trouvés sur la page 1: 14
Scraping page 2
Nombre de produits trouvés sur la page 2: 30
Scraping page 3
Nombre de produits trouvés sur la page 3: 29
Scraping page 4
Nombre de produits trouvés sur la page 4: 30
Scraping page 5
Nombre de produits trouvés sur la page 5: 30
Scraping page 6
Nombre de produits trouvés sur la page 6: 30
Scraping page 7
Nombre de produits trouvés sur la page 7: 30
Scraping page 8
Nombre de produits trouvés sur la page 8: 30
Scraping page 9
Nombre de produits trouvés sur la page 9: 30
Scraping page 10
Nombre de produits trouvés sur la page 10: 30
Scraping page 11
Nombre de produits trouvés sur la page 11: 30
Scraping page 12
Nombre de produits trouvés sur la page 12: 30
Scraping page 13
Nombre de produits trouvés sur la page 13: 30
Scraping page 14
Nombre de produits trouvés sur la page 14: 30
Scraping page 15
Nombre de produits trouvés sur la page 15: 4
Scraping page 16
Nombre de produits trouvés sur la page 16: 9
Erreur lors 

In [9]:
df = pd.DataFrame({
    'Nom': product_names,
    'Prix': product_prices,
    'Unité': product_units,
    'Réduction': discount_percentages,
    
})

df

,Nom,Prix,Unité,Réduction
0,KREMA Red Dingue Bonbons aux fruits rouges,"5,32€\n3,51€",580g,None
1,HARIBO Dragibus aux couleurs d'Halloween,"2,31€",250g,3 achetés = 2 payés
2,AUCHAN LE CHARCUTIER Chiffonnade de Coppa,"3,69€",80g,-50% sur le 2ème
3,AUCHAN LE CHARCUTIER Chiffonnade de jambon ser...,"3,59€",80g,-50% sur le 2ème
4,AUCHAN LE CHARCUTIER Chiffonnade de rosette,"3,59€",80g,-50% sur le 2ème
...,...,...,...,...
8973,Bananes Prix Bas,"0,99€",5 pièces,None
8974,AUCHAN CULTIVONS LE BON thon listao entier au ...,"2,23€",140g,None
8975,POUCE Œufs de poules,"1,03€",6 œufs,None
8976,POKEMON Mug Pikachu 320ml - Noir,"14,00€",None,None


In [11]:
# Sauvegarder les données dans un fichier Excel
df.to_csv("auchan_products.csv", index=False)

print("Données enregistrées dans auchan_products.csv")

Données enregistrées dans auchan_products.csv
